In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

In [2]:
properties = pd.read_csv("property.csv")
user_activity = pd.read_csv("user_activity.csv")
event_types = pd.read_excel("event_types.xlsx")

In [3]:
properties.head()

,item_id,deposit,monthly_rent,district_uuid,room_qty,unit_area,has_elevator,building_floor_count,unit_floor,has_storage_area,property_age
0,91c0e569-bddd-4128-9720-2550bb85580e,64800000.0,0,263682f6-d0cd-4569-aeec-e727b76b7665,1.0,42,False,3.0,0.0,True,23.0
1,b00b7919-06be-4d26-98b8-1971787e1d46,72000000.0,4320000,97c9535e-3985-47ce-a84c-a962c838a76b,2.0,116,True,NaN,1.0,True,16.0
2,9eddb6bc-e424-4774-b55f-bfd54366d627,50400000.0,1440000,b790f536-c274-4147-86e0-94d9b6d7352d,1.0,74,False,2.0,0.0,True,19.0
3,12cf6b07-5d56-4126-94d2-ce9cbfe2214f,36000000.0,864000,93d06676-4975-4cc5-919b-3a0c29c7ad43,1.0,60,True,NaN,2.0,False,6.0
4,929eb20c-3694-46b2-b96c-91117b995d1b,28800000.0,1296000,58e59fa9-9947-478f-9cef-bc6a2cbe49a9,1.0,45,True,NaN,1.0,True,4.0


In [4]:
user_activity.head()

,item_id,user_id,event_type,create_timestamp
0,00062bc5-2535-4b1e-bbcb-228526c990b8,182aa519-83a8-848f-84a1-8697046d84c2,seen,2020-02-03 15:47:25.273977
1,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,seen,2020-02-04 20:19:31.040304
2,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,sent_catalog_link,2020-02-04 20:19:00.110416
3,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,visit_request-canceled,2020-02-04 20:54:31.595305
4,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,visit_request-new,2020-02-04 20:20:15.918646


In [5]:
event_types

,Step,Event type,meaning
0,1,seen_in_list,Home impression on website
1,2,seen,Home view on Address website
2,3,suggest-new,"Home link sent via text message, suggested by ..."
3,4,suggest_similar,Similar homes sent to tenant
4,5,sent_catalog_link,Home link sent via text message
5,6,visit_request-new,User requested to visit the home
6,7,visit_request-canceled,Request to visit the home was cancelled
7,8,visit-new,Home visit scheduled
8,9,visit-canceled,Home visit canceled after being scheduled
9,10,visit-unsuccess,Home visit done but the tenant did not liked t...


In [6]:
user_activity.item_id.nunique()

4930

In [7]:
user_activity.user_id.nunique()

22826

In [8]:
user_activity.shape

(323893, 4)

In [9]:
user_activity.drop('create_timestamp',1,inplace=True)

/var/folders/dm/s7p9m_4j3bgfqrvz0vd81ss40000gn/T/ipykernel_1411/2509693123.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_activity.drop('create_timestamp',1,inplace=True)


In [10]:
user_activity.head()

,item_id,user_id,event_type
0,00062bc5-2535-4b1e-bbcb-228526c990b8,182aa519-83a8-848f-84a1-8697046d84c2,seen
1,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,seen
2,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,sent_catalog_link
3,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,visit_request-canceled
4,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,visit_request-new


In [11]:
len(user_activity.event_type.value_counts())

18

In [12]:
event_types.shape

(18, 3)

In [13]:
events_dict = event_types[['Step',"Event type"]].rename(columns={"Step":"step","Event type":"event_type"}).to_dict('record')
events = {}
for event in events_dict:
    events[event['event_type']] = event['step']
    # print(event['step'])
events

/var/folders/dm/s7p9m_4j3bgfqrvz0vd81ss40000gn/T/ipykernel_1411/165590176.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  events_dict = event_types[['Step',"Event type"]].rename(columns={"Step":"step","Event type":"event_type"}).to_dict('record')


{'seen_in_list': 1,
 'seen': 2,
 'suggest-new': 3,
 'suggest_similar': 4,
 'sent_catalog_link': 5,
 'visit_request-new': 6,
 'visit_request-canceled': 7,
 'visit-new': 8,
 'visit-canceled': 9,
 'visit-unsuccess': 10,
 'visit-success': 11,
 'meeting_request-new': 12,
 'meeting_request-canceled': 13,
 'meeting-new': 14,
 'meeting-canceled': 15,
 'meeting-unsuccess': 16,
 'meeting-success': 16,
 'deal-success': 18}

In [14]:
user_activity.event_type = user_activity.event_type.map(events)

In [15]:
user_activity.head()

,item_id,user_id,event_type
0,00062bc5-2535-4b1e-bbcb-228526c990b8,182aa519-83a8-848f-84a1-8697046d84c2,2
1,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,2
2,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,5
3,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,7
4,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,6


In [16]:
user_activity.rename(columns={"event_type":"rating"},inplace=True)

In [17]:
user_activity.head()

,item_id,user_id,rating
0,00062bc5-2535-4b1e-bbcb-228526c990b8,182aa519-83a8-848f-84a1-8697046d84c2,2
1,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,2
2,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,5
3,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,7
4,00062bc5-2535-4b1e-bbcb-228526c990b8,189a081a-ae0f-499d-9092-01758d93fa7f,6


In [18]:
user_activity.groupby('user_id').sum()

/var/folders/dm/s7p9m_4j3bgfqrvz0vd81ss40000gn/T/ipykernel_1411/2001468288.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  user_activity.groupby('user_id').sum()


,rating
user_id,
0000c6f3-2520-4dc7-9e53-3117f02aedaa,2
00037d8c-8294-892d-89f9-81f08927e249,4
0004ca5d-8289-8a63-8ac7-847e622f0810,2
0009a855-8276-8ec1-8659-829c6c865eda,12
0009af04-87f4-8387-8e7c-8ffa22fce053,8
...,...
fff3d04f-8736-8ed7-8c78-8a25e2615d84,8
fff51a2d-0a28-4838-a130-f8a9a0df3ee7,3
fff8a21c-8625-8513-86f2-866e90c1a303,18


In [19]:
user_activity[user_activity['user_id']=="fff8a21c-8625-8513-86f2-866e90c1a303"]

,item_id,user_id,rating
2072,0155946b-d6dd-43b8-b5d4-9ef996be3016,fff8a21c-8625-8513-86f2-866e90c1a303,2
49830,292117b9-9b7d-4c39-855d-07e4cfdbdfe5,fff8a21c-8625-8513-86f2-866e90c1a303,2
67436,36c496b6-7056-44f6-976b-963db5263b66,fff8a21c-8625-8513-86f2-866e90c1a303,2
110881,5e24995e-e685-44b1-9be9-f449e5647bb7,fff8a21c-8625-8513-86f2-866e90c1a303,2
140674,730c20b1-d65f-4c7b-b3f4-10fc45f1391b,fff8a21c-8625-8513-86f2-866e90c1a303,2
168655,8795f735-e137-4dbe-85c7-7beb6e3256b7,fff8a21c-8625-8513-86f2-866e90c1a303,2
172836,8b826c59-bc8e-4463-8d77-effbe4632f9f,fff8a21c-8625-8513-86f2-866e90c1a303,2
214725,b52f7d19-672e-44ec-bb6e-666bb81c88bc,fff8a21c-8625-8513-86f2-866e90c1a303,2
260648,d088bbe4-d5ff-4eb3-bb2e-23ccc2b3171d,fff8a21c-8625-8513-86f2-866e90c1a303,2


In [20]:
def get_sparse_pivot(data,index,columns,values):
    
    row_c = CategoricalDtype(sorted(data[index].unique()), ordered=True)
    col_c = CategoricalDtype(sorted(data[columns].unique()), ordered=True)

    row = data[index].astype(row_c).cat.codes
    col = user_activity[columns].astype(col_c).cat.codes
    sparse_matrix = csr_matrix((data[values], (row, col)), \
                               shape=(row_c.categories.size, col_c.categories.size))
    
    return sparse_matrix

In [21]:
user_item_sparse = get_sparse_pivot(user_activity,index = "user_id",columns = "item_id", values = "rating")